In [1]:
from utils.utils import *


In [3]:
def read_image(image_path):
    """
    :param image_path: String, path to the input image.
    Returns:
        image: PIL Image.
    """
    image = Image.open(image_path).convert('RGB')
    return image

def ocr(image, processor, model, generation_length, device):
    pixel_values = processor(image, return_tensors='pt').pixel_values.to(device)
    generated_ids = model.generate(pixel_values, max_length=generation_length)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

def eval_new_data(image=None, model=None, processor=None, device='cpu'):
    generation_length = 12
    text = ocr(image, processor, model, generation_length, device)
    return text



In [4]:
#### Function for MP utils
def easy_ocr_reader_sequential(image:np.ndarray)->str:
    ocr_text=[]
    # enhanced_img=enhance_image(image)
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image)
    for (bbox, text, prob) in result:
        if prob > 0:
            ocr_text.append(text)
    if ocr_text!=[]:
       return ocr_text[0]
    else:
       return '' 
  
def tesserac_ocr_reader_sequential(image:np.ndarray)->str:
    text = pytesseract.image_to_string(image,lang='eng')
    return text
    
def trocr_processor_sequential(img, model, processor, device='cuda'):
    """
    Process the image using a preloaded model and processor.
    """
    ocr_text = eval_new_data(image=img, model=model, processor=processor, device=device)
    return ocr_text
    

In [5]:
### Blow function is to compare the output from nocr systems 
from collections import Counter

def compare_strings(strings):
    if not strings:
        return ""
    
    # Find the maximum length of the strings
    max_length = max(len(s) for s in strings)
    
    final_string = []
    
    for i in range(max_length):
        # Collect all crereharacters at the current position from all strings
        chars_at_pos = [s[i] for s in strings if i < len(s)]
        
        # Find the most common character at this position
        if chars_at_pos:
            most_common_char, _ = Counter(chars_at_pos).most_common(1)[0]
            final_string.append(most_common_char)
        else:
            final_string.append(' ')
    
    return ''.join(final_string)

# # Example usage:
# strings = [
#     "hello world",
#     "hfllo worle",
#     "hello worlz",
#     "hello worle"
# ]

# final_output = compare_strings(strings)
# print(final_output)


In [15]:
def remove_non_alphanumeric(input_string):
    pattern = r'[^a-zA-Z0-9]+'
    result = re.sub(pattern, '', input_string)
    return result

def run_ocr(image:np.ndarray,tr_model,tr_device,tr_processor)->str:
    es_ocr_result=easy_ocr_reader_sequential(image)
    tes_ocr_result=tesserac_ocr_reader_sequential(image)
    tr_ocr_result=trocr_processor_sequential(image,model=tr_model,device=tr_device,processor=tr_processor)
    return [remove_non_alphanumeric(es_ocr_result),remove_non_alphanumeric(tes_ocr_result),remove_non_alphanumeric(tr_ocr_result)]

def run_ocr_mt(image:np.ndarray,tr_model,tr_device,tr_processor)->str:
    main_queue=multiprocessing.Queue()
    strings_=[]
    t1=threading.Thread(target=easy_ocr_reader,args=[image,main_queue])
    t2=threading.Thread(target=tesserac_ocr_reader,args=[image,main_queue])
    t3=threading.Thread(target=easy_ocr_reader,args=[image,tr_model,tr_device,tr_processor,main_queue])
    t1.start()
    t2.start()
    t3.start()
    t1.join()
    t2.join()
    t3.join()
    while not main_queue.empty():
        strings_.append(main_queue.get())
    
    return strings_

In [4]:
def tr_ocr_model_loader(numthreads=3,cuda_on=True,model_name='microsoft/trocr-small-printed',device_number=0):
    if cuda_on:
        device = f'cuda:{device_number}' if torch.cuda.is_available() else 'cpu'
    else:
        device='cpu'
    model_json_list=[]
    for _ in range(numthreads):
        model_dict={}
        model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)
        processor = TrOCRProcessor.from_pretrained(model_name)
        # Loading params into temp json
        model_dict['device']=device
        model_dict['model_name']=model_name
        model_dict['model']=model
        model_dict['processor']=processor
        model_json_list.append(model_dict)
    return model_json_list

In [8]:
def ocr_processor_mp_phase_three(folder_paths,num_thread=3):
        all_process_list=[]
        chunks=list_divider(folder_paths,number_of_part=num_thread)    
        tr_ocr_models=tr_ocr_model_loader(numthreads=num_thread)
        # main_queue=multiprocessing.Queue()
        current_date=get_current_date()
        csv_file_name=f"{current_date}_ocr_results.csv"
        csv_file_path=os.path.join(os.getcwd(),csv_file_name)
        for chunk,model_json in zip(chunks,tr_ocr_models):
            t=threading.Thread(target=ocr_sub_process_phase_three,args=[folder_paths,csv_file_name,model_json['model'],model_json['processor'],model_json['device']])
            t.start()
            all_process_list.append(t)
        for process in all_process_list:
            process.join()
        return csv_file_path
    
        
            

In [9]:
def get_current_date():
  """
  date_time_utils
  Returns the current date in DD-MM-YYYY format.
  """
  today = date.today()
  return today.strftime("%d_%m_%Y")

In [10]:
def file_writer(comma_string,file_name,mode='append'):
    """
    file_utils
    Modes: 
        1. append : Write while also keeping the existing content (Default)
        2. write  : Overwrites the existing content.
    """
    mode_string='a'
    if file_name.endswith('.csv'):
        file_name=file_name
    else:    
        file_name=f"{file_name}.csv"
        
    if os.path.isfile(file_name):
        if mode =='append':
            mode_string='a+'
    else:
        if mode =='overwrite':
            mode_string='w+'
        
    with open(f'{file_name}',f'{mode_string}') as file:
        file.write(f'{comma_string}\n')
        file.close()

In [11]:
def ocr_sub_process_phase_three(folder_paths,csv_file_name,tr_model,tr_processor,tr_device):
        for path in folder_paths:
            file_list=sort_files_by_name(path)
            idx=path.split('/')[-1]
            
            for image_path in file_list:
                
                img = cv2.imread(image_path)
                #Currently we write each and every possiblity in the csv file for each Possible ID
                strings=run_ocr(img,tr_model,tr_device,tr_processor)
                final_string=compare_strings(strings)
                final_csv_string=f'{idx},{final_string}'
                file_writer(final_csv_string,csv_file_name)

#### Create function to get all string found for that id and then compare them the most repeated string will be selected as number plate

In [12]:
#### Function for MP utils
def easy_ocr_reader_td(image:np.ndarray,queue=None)->str:
    ocr_text=[]
    # enhanced_img=enhance_image(image)
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image)
    for (bbox, text, prob) in result:
        if prob > 0:
            ocr_text.append(text)
    if ocr_text!=[]:
        queue.put(ocr_text[0])
    else:
        queue.put("")

def tesserac_ocr_reader_td(image:np.ndarray,queue=None)->str:
    text = pytesseract.image_to_string(image,lang='eng')
    queue.put(text)
    
def trocr_processor_td(img, model, processor,queue,device='cuda'):
    """
    Process the image using a preloaded model and processor.
    """
    ocr_text = eval_new_data(image=img, model=model, processor=processor, device=device)
    queue.put(ocr_text)
    

In [37]:
with open('test_file.csv','a+') as f:
    f.close()

In [41]:
current_date=get_current_date()
csv_file_name=f"{current_date}_lp_ocr.csv"
csv_file_path=os.path.join(os.getcwd(),csv_file_name)
print(csv_file_path)    

/home/hlink/workspace/learning/boxmot/24_05_2024_lp_ocr.csv


In [2]:
from utils.utils import *

In [3]:
test_paths=['/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/74',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/57',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/60',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/92',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/139',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/364',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/479',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/483',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/464',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/469',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/475',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/366',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/381',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/406',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/407',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/410',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/299',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/307',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/308',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/335',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/340',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/257',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/260',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/280',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/3',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/110',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/111',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/120',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/153',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/127',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/513',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/520',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/528',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/545',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/556',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/99',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/196',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/4',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/38',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/35',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/48',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/133',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/166',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/411',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/433',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/448',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/449',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/463',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/202',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/157',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/363',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/375',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/180',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/94',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/557',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/140',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/71',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/175',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/177',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/200',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/216',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/228',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/156',
 '/home/hlink/workspace/learning/boxmot/24_05_2024_infenrence_output/threashold_lp_images/233']


In [4]:
%%time
ocr_processor_mp_phase_three(folder_paths=test_paths,num_thread=3)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/hlink/workspace/learning/boxmot

CPU times: user 4min 25s, sys: 1min 33s, total: 5min 59s
Wall time: 4min 56s


'/home/hlink/workspace/learning/boxmot/24_05_2024_ocr_results.csv'

add fall back images 
Also in csv file add liscense plate image path for each ocr process done so that in future when we retirve data we can 

directly provide image path in the json file tooo